In [1]:
import pandas as pd
import numpy as np

In [2]:
assembly_data = pd.read_csv('transposed_report.tsv', sep='\t')

In [3]:
assembly_data['k'] = assembly_data['Assembly'].apply(lambda x: x.split('_')[0][1:])

In [10]:
assembly_data['coverage'] = assembly_data['Assembly'].apply(lambda x: x.split('_')[2][:len(x.split('_')[2])-1])

In [11]:
assembly_data

,Assembly,# contigs (>= 0 bp),# contigs (>= 1000 bp),# contigs (>= 5000 bp),# contigs (>= 10000 bp),# contigs (>= 25000 bp),# contigs (>= 50000 bp),Total length (>= 0 bp),Total length (>= 1000 bp),Total length (>= 5000 bp),...,NA50,NGA50,NA75,NGA75,LA50,LGA50,LA75,LGA75,k,coverage
0,k30_ecoli_100X_1-contigs,574,149,111,96,57,27,4600795,4559642,4456040,...,58688,57958,33607,31903,21,22,47,49,30,100
1,k36_ecoli_100X_1-contigs,435,129,101,83,54,32,4607285,4569436,4494389,...,74998,73690,42312,42262,19,20,40,41,36,100
2,k42_ecoli_100X_1-contigs,376,119,91,81,57,31,4617624,4586981,4510894,...,82877,82877,42318,42262,18,18,38,39,42,100
3,k48_ecoli_100X_1-contigs,339,109,88,77,54,30,4621162,4591434,4533092,...,96181,95861,44790,42324,16,17,35,36,48,100
4,k54_ecoli_100X_1-contigs,295,103,85,74,53,32,4623081,4595244,4546923,...,95957,95906,48889,48252,16,17,33,34,54,100
5,k60_ecoli_100X_1-contigs,284,98,80,69,51,32,4628369,4599433,4550881,...,106366,106366,53796,53764,15,15,31,32,60,100
6,k66_ecoli_100X_1-contigs,264,97,78,68,51,32,4628447,4599865,4547044,...,106378,106378,53927,53927,15,15,31,31,66,100
7,k72_ecoli_100X_1-contigs,257,92,74,64,50,31,4634937,4604631,4551776,...,113967,112701,58833,58833,14,15,29,29,72,100
8,k78_ecoli_100X_1-contigs,242,90,74,65,51,32,4634528,4604129,4556604,...,113390,106402,58833,58233,14,15,29,30,78,100
9,k84_ecoli_100X_1-contigs,225,89,72,65,51,32,4632976,4604178,4556104,...,113390,106417,58239,58239,14,15,30,30,84,100


In [12]:
reads = assembly_data['Assembly'].apply(lambda x: x.strip('-contigs'))

In [13]:
reads = np.unique(reads)
reads_data = pd.DataFrame(index=reads)

In [18]:
reads_data.to_csv('temp.tsv', sep='\t')

'ecoli_20X_7-contigs 1'